In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df=pd.read_csv('Advertising.csv')

In [3]:
X=df.drop('sales', axis=1)
y=df['sales']

In [4]:
from sklearn.preprocessing import PolynomialFeatures

In [5]:
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [6]:
poly_features=polynomial_converter.fit_transform(X)

In [10]:
print(X.shape)
poly_features.shape

(200, 3)


(200, 19)

In [11]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [21]:
X_train.shape

(140, 19)

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
scaler = StandardScaler()

In [24]:
scaler.fit(X_train)

StandardScaler()

In [25]:
X_train = scaler.transform(X_train)

In [26]:
X_test = scaler.transform(X_test)

In [27]:
X_train[0]

array([ 0.49300171, -0.33994238,  1.61586707,  0.28407363, -0.02568776,
        1.49677566, -0.59023161,  0.41659155,  1.6137853 ,  0.08057172,
       -0.05392229,  1.01524393, -0.36986163,  0.52457967,  1.48737034,
       -0.66096022, -0.16360242,  0.54694754,  1.37075536])

In [29]:
# I Valori in poly_features sono molto grandi, quelli in X_train sono stati
# scalati usando media e deviazione std
poly_features[0]

array([2.30100000e+02, 3.78000000e+01, 6.92000000e+01, 5.29460100e+04,
       8.69778000e+03, 1.59229200e+04, 1.42884000e+03, 2.61576000e+03,
       4.78864000e+03, 1.21828769e+07, 2.00135918e+06, 3.66386389e+06,
       3.28776084e+05, 6.01886376e+05, 1.10186606e+06, 5.40101520e+04,
       9.88757280e+04, 1.81010592e+05, 3.31373888e+05])

In [32]:
# RIDGE REGRESSION - è un metodo di regolarizzazione
# consiste nell'aggiungere un termine beta^2 alla somma dei quadrati degli errori
# ha anche un lambda tunabile (in sk learn è alpha):
# ERRORE = somma (y_i - beta_0 - somma(beta_j*x_i,j))^2 + L*somma(beta_j^2)
# Questo metodo previene l'overfitting

In [33]:
from sklearn.linear_model import Ridge

In [36]:
#  Minimizes the objective function::
#|  
#|  ||y - Xw||^2_2 + alpha * ||w||^2_2
#|  
#|  This model solves a regression model where the loss function is
#|  the linear least squares function and regularization is given by
#|  the l2-norm. Also known as Ridge Regression or Tikhonov regularization.
#|  This estimator has built-in support for multi-variate regression
#|  (i.e., when y is a 2d-array of shape (n_samples, n_targets)).

In [37]:
ridge_model= Ridge(alpha=10)

In [38]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [39]:
test_predictions = ridge_model.predict(X_test)

In [40]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [41]:
MAE = mean_absolute_error(y_test, test_predictions)

In [42]:
MAE

0.5774404204714171

In [43]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [44]:
RMSE

0.8946386461319646

In [45]:
# Come faccio a sapere se alpha=10 va bene? Non lo so -> Cross validation

In [46]:
from sklearn.linear_model import RidgeCV

In [49]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0))

In [50]:
#Se non specifico niente, funziona con leave-one-out
#quindi X_train diventa sia train che test
ridge_cv_model.fit(X_train,y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]))

In [51]:
#alpha che performa meglio?
ridge_cv_model.alpha_

0.1

In [52]:
from sklearn.metrics import SCORERS

In [53]:
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [54]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0), scoring = 'neg_mean_absolute_error')

In [55]:
ridge_cv_model.fit(X_train,y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [56]:
test_predictions = ridge_cv_model.predict(X_test)

In [57]:
MAE = mean_absolute_error(y_test, test_predictions)

In [58]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [59]:
MAE

0.42737748843333073

In [60]:
RMSE

0.6180719926923646

In [61]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

In [62]:
#LASSO REGRESSION
# Least Absolute Shrinkage & Selection Operator
#Anzichè aggiungere all'errore la somma dei quadrati dei coeff
#Aggiunge la somma dei valori assoluti dei coeff (sempre con alpha)


In [64]:
from sklearn.linear_model import LassoCV

In [65]:
lasso_cv_model = LassoCV(eps=0.001, n_alphas=100, cv=5)

In [67]:
lasso_cv_model.fit(X_train,y_train)

C:\Users\00006258\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.870e+00, tolerance: 3.684e-01
  model = cd_fast.enet_coordinate_descent(


LassoCV(cv=5)

In [68]:
#dato che non converge posso aumentare epsilon o il numero di iter
lasso_cv_model = LassoCV(eps=0.001, n_alphas=100, cv=5, max_iter=1000000)

In [69]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, max_iter=1000000)

In [70]:
lasso_cv_model = LassoCV(eps=0.1, n_alphas=100, cv=5)

In [71]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.1)

In [73]:
lasso_cv_model.alpha_

0.4943070909225832

In [75]:
test_predictions=lasso_cv_model.predict(X_test)

In [76]:
 MAE = mean_absolute_error(y_test, test_predictions)

In [77]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [78]:
MAE

0.6541723161252854

In [79]:
RMSE

1.1308001022762533

In [80]:
#L'errore è peggiorato
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [83]:
#performa peggio perchè sta usando solo due features ->è facile da interpretare
#questo potrebbe dirmi che in un database grosso dovrei considerare solo queste

In [86]:
#ELASTIC NET: combina LASSO e Ridge
#L'idea è che con L1 (Lasso) si vuole imporre Somma|beta|<s
# e con L2 (ridge) si vuole imporre Somma(beta^2)<s
#ELASTIC NET mette nella stessa espressione le due somme pesate con alpha1 e alpha2

In [87]:
from sklearn.linear_model import ElasticNetCV

In [88]:
elastic_model = ElasticNetCV(l1_ratio=[.1,.5,.7,.9,.95,.99,1], eps=0.001, n_alphas=100, max_iter=1000000)

In [89]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [94]:
elastic_model.l1_ratio_
#Questo significa che il modello è come se usasse solo LASSO

1.0

In [95]:
elastic_model.alpha_

0.004943070909225833

In [96]:
elastic_model.coef_

array([ 4.86023329,  0.12544598,  0.20746872, -4.99250395,  4.38026519,
       -0.22977201, -0.        ,  0.07267717, -0.        ,  1.77780246,
       -0.69614918, -0.        ,  0.12044132, -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ])

In [100]:
test_predictions=elastic_model.predict(X_test)

In [101]:
 MAE = mean_absolute_error(y_test, test_predictions)

In [102]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [103]:
MAE

0.43350346185900623

In [104]:
RMSE

0.6063140748984029

In [105]:
#In generale ha senso usare direttamente l'elastic net dato che se si usa
# un range di alpha adeguato è come testare ridge e lasso insieme